In [1]:
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
district_true = school_data_complete['type'] == 'District'
district_data = school_data_complete[district_true]
district_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
# Make new dataframe and populate it with corresponding values
district_summary= pd.DataFrame([0])

# Calculate the total number of schools
# Calculate the total number of students
district_summary["Number of Schools"] = len(district_data['School ID'].value_counts())
district_summary["Number of Students"] = district_data['Student ID'].count()

# Calculate the total budget
budget_vals = district_data['budget'].unique()
district_summary["Total Budget"] = budget_vals.sum()

# Calculate the average math score
math_score = district_data["math_score"]
district_summary["Average Math Score"] = math_score.mean()

# Calculate the average reading score
reading_score = district_data["reading_score"]
district_summary["Average Reading Score"] = reading_score.mean()

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
district_summary["Overall Average Score"] = (reading_score + math_score)/2

# Calculate the percentage of students with a passing math score (70 or greater)
math_score = district_data["math_score"]
district_summary["% Passing Math"] = (math_score >= 70).mean() * 100

# Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading_score = district_data["reading_score"]
district_summary["% Passing Reading"] = (passing_reading_score >= 70).mean() * 100

district_summary = district_summary.drop([0], axis=1)

In [4]:
district_summary

,Number of Schools,Number of Students,Total Budget,Average Math Score,Average Reading Score,Overall Average Score,% Passing Math,% Passing Reading
0,7,26976,17347923,76.987026,80.962485,72.5,66.518387,80.905249


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [5]:
# Create an overview table that summarizes key metrics about each school, including:
schools_summary= school_data_complete.drop(columns=['Student ID','student_name', 'gender', 'grade', 'School ID'])

schools_summary = schools_summary.groupby(['school_name', 'type']).mean()

schools_summary = schools_summary.reset_index(drop=False)

schools_summary = schools_summary.set_index('school_name')



# Total Students
# Total School Budget
# Per Student Budget
# Average Reading Score
schools_summary = schools_summary.rename(columns={"type": "School Type", "reading_score" : "Average Reading Score", "math_score"
                                                 : "Average Math Score", "size": "Total Students", "budget": "Total School Budget"})
budget = schools_summary['Total School Budget'].values
students = schools_summary['Total Students'].values
schools_summary['Per Student Budget'] = budget/students

# % Passing Math
schools_summary2 = school_data_complete
passing_math = school_data_complete.loc[schools_summary2['math_score']>69,:]

passing_math = passing_math.groupby('school_name').math_score.count().reset_index()

passing_math = passing_math.rename(columns={"math_score":"% Passing Math"})

# Merge the two dataframes
schools_summary = passing_math.merge(schools_summary, on="school_name")
schools_summary['% Passing Math'] = (schools_summary['% Passing Math'] / schools_summary['Total Students']) * 100


# % Passing Reading
schools_summary2 = school_data_complete
passing_reading = school_data_complete.loc[schools_summary2['reading_score']>69,:]

passing_reading = passing_reading.groupby('school_name').reading_score.count().reset_index()

passing_reading = passing_reading.rename(columns={"reading_score":"% Passing Reading"})

schools_summary = passing_reading.merge(schools_summary, on="school_name")

schools_summary['% Passing Reading'] = (schools_summary['% Passing Reading'] / schools_summary['Total Students']) * 100

# Overall Passing Rate (Average of the above two)

schools_summary['% Overall Passing'] = (schools_summary['% Passing Math'] + schools_summary['% Passing Reading']) / 2

schools_summary = schools_summary.set_index('school_name')

schools_summary = schools_summary.rename_axis("")

schools_summary

,% Passing Reading,% Passing Math,School Type,Average Reading Score,Average Math Score,Total Students,Total School Budget,Per Student Budget,% Overall Passing
,,,,,,,,,
Bailey High School,81.933280,66.680064,District,81.033963,77.048432,4976.0,3124928.0,628.0,74.306672
Cabrera High School,97.039828,94.133477,Charter,83.975780,83.061895,1858.0,1081356.0,582.0,95.586652
Figueroa High School,80.739234,65.988471,District,81.158020,76.711767,2949.0,1884411.0,639.0,73.363852
Ford High School,79.299014,68.309602,District,80.746258,77.102592,2739.0,1763916.0,644.0,73.804308
Griffin High School,97.138965,93.392371,Charter,83.816757,83.351499,1468.0,917500.0,625.0,95.265668
Hernandez High School,80.862999,66.752967,District,80.934412,77.289752,4635.0,3022020.0,652.0,73.807983
Holden High School,96.252927,92.505855,Charter,83.814988,83.803279,427.0,248087.0,581.0,94.379391
Huang High School,81.316421,65.683922,District,81.182722,76.629414,2917.0,1910635.0,655.0,73.500171
Johnson High School,81.222432,66.057551,District,80.966394,77.072464,4761.0,3094650.0,650.0,73.639992


## Top Performing Schools (By Passing Rate)
* Sort and display the top five schools in overall passing rate

In [6]:
top_schools = schools_summary.sort_values(by='% Overall Passing', ascending=False).head()
top_schools = top_schools.rename_axis("")
top_schools

,% Passing Reading,% Passing Math,School Type,Average Reading Score,Average Math Score,Total Students,Total School Budget,Per Student Budget,% Overall Passing
,,,,,,,,,
Cabrera High School,97.039828,94.133477,Charter,83.975780,83.061895,1858.0,1081356.0,582.0,95.586652
Thomas High School,97.308869,93.272171,Charter,83.848930,83.418349,1635.0,1043130.0,638.0,95.290520
Pena High School,95.945946,94.594595,Charter,84.044699,83.839917,962.0,585858.0,609.0,95.270270
Griffin High School,97.138965,93.392371,Charter,83.816757,83.351499,1468.0,917500.0,625.0,95.265668
Wilson High School,96.539641,93.867718,Charter,83.989488,83.274201,2283.0,1319574.0,578.0,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [7]:
bottom_schools = schools_summary.sort_values(by='% Overall Passing', ascending=True).head()
bottom_schools = bottom_schools.rename_axis("")
bottom_schools

,% Passing Reading,% Passing Math,School Type,Average Reading Score,Average Math Score,Total Students,Total School Budget,Per Student Budget,% Overall Passing
,,,,,,,,,
Rodriguez High School,80.220055,66.366592,District,80.744686,76.842711,3999.0,2547363.0,637.0,73.293323
Figueroa High School,80.739234,65.988471,District,81.158020,76.711767,2949.0,1884411.0,639.0,73.363852
Huang High School,81.316421,65.683922,District,81.182722,76.629414,2917.0,1910635.0,655.0,73.500171
Johnson High School,81.222432,66.057551,District,80.966394,77.072464,4761.0,3094650.0,650.0,73.639992
Ford High School,79.299014,68.309602,District,80.746258,77.102592,2739.0,1763916.0,644.0,73.804308


## Math Scores By Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
# Create a table that displays each school's math grade by grade level
math_scores_by_grade = school_data_complete.drop(columns=['Student ID','student_name', 'gender', 'School ID', 'size', 'budget', 'reading_score'])

# Find averages
math_scores_by_grade = math_scores_by_grade.groupby(['school_name', 'grade']).mean()

# Reset index to make it more clear
math_scores_by_grade = math_scores_by_grade.reset_index(drop=False)
math_scores_by_grade = math_scores_by_grade.set_index('school_name')

# Pivot table to display grade index as columns
math_scores_by_grade = math_scores_by_grade.pivot(columns='grade', values='math_score')
math_scores_by_grade = math_scores_by_grade.rename_axis("", axis=0)
math_scores_by_grade = math_scores_by_grade.rename_axis("", axis=1)

math_scores_by_grade

,10th,11th,12th,9th
,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [9]:
# Create a table that displays each school's reading grade by grade level
reading_scores_by_grade = school_data_complete.drop(columns=['Student ID','student_name', 'gender', 'School ID', 'size', 'budget', 'math_score'])

# Find averages
reading_scores_by_grade = reading_scores_by_grade.groupby(['school_name', 'grade']).mean()

# Reset index to make it more clear
reading_scores_by_grade = reading_scores_by_grade.reset_index(drop=False)
reading_scores_by_grade = reading_scores_by_grade.set_index('school_name')

# Pivot table to display grade index as columns
reading_scores_by_grade = reading_scores_by_grade.pivot(columns='grade', values='reading_score')
reading_scores_by_grade = reading_scores_by_grade.rename_axis("", axis=0)
reading_scores_by_grade = reading_scores_by_grade.rename_axis("", axis=1)
reading_scores_by_grade


,10th,11th,12th,9th
,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714


## Scores by School Spending
* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [10]:
school_spending = schools_summary[['Average Math Score', 'Average Reading Score', '% Passing Reading', '% Passing Math', '% Overall Passing', 'Per Student Budget']]

In [11]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [12]:
school_spending["Spending Ranges (Per Student)"] = pd.cut(school_spending["Per Student Budget"], spending_bins, labels=group_names)
school_spending = school_spending.drop(columns=['Per Student Budget'])
school_spending = school_spending.groupby(school_spending["Spending Ranges (Per Student)"], as_index=True)
# school_spending = school_spending.set_index('Spending Ranges (Per Student)').mean()
school_spending.mean()

C:\Users\megam\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,96.610877,93.460096,95.035486
$585-615,83.599686,83.885211,95.900287,94.230858,95.065572
$615-645,79.079225,81.891436,86.106569,75.668212,80.887391
$645-675,76.997210,81.027843,81.133951,66.164813,73.649382


## Scores by School Size
* Perform the same operations as above, based on school size.

In [13]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [28]:
school_size = schools_summary[['Average Math Score', 'Average Reading Score', '% Passing Reading', '% Passing Math', '% Overall Passing', 'Total Students']]

In [15]:
school_size["Size"] = pd.cut(school_size["Total Students"], size_bins, labels=group_names)
school_size = school_size.drop(columns=['Total Students'])
school_size = school_size.groupby(school_size["Size"], as_index=True)
# school_size = school_size.set_index('Total Students').mean()
school_size.mean()

C:\Users\megam\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,% Overall Passing
Size,,,,,
Small (<1000),83.821598,83.929843,96.099437,93.550225,94.824831
Medium (1000-2000),83.374684,83.864438,96.790680,93.599695,95.195187
Large (2000-5000),77.746417,81.344493,82.766634,69.963361,76.364998


## Scores by School Type
* Perform the same operations as above, based on school type.

In [20]:
schools_summary = schools_summary.rename_axis("school_name", axis=1)
schools_summary = schools_summary.reset_index()
school_type = schools_summary[['Average Math Score', 'Average Reading Score', '% Passing Reading', '% Passing Math', '% Overall Passing', 'school_name']]

In [48]:
df_to_merge = school_data_complete.drop(columns=['Student ID','student_name', 'gender', 'size', 'School ID', 'budget', 'reading_score', 'grade', 'math_score'])


school_type = schools_summary.merge(df_to_merge, on='school_name', how='inner', copy=False)
school_type = school_type.drop(columns=['index', 'Total Students', 'Total School Budget', 'School Type'] )
school_type = school_type.drop_duplicates()
school_type = school_type.reset_index(drop=True)
school_type
# Classification
school_type = school_type.groupby(school_type["type"], as_index=True).mean()
school_type = school_type.rename_axis("", axis=0)
school_type

,% Passing Reading,% Passing Math,Average Reading Score,Average Math Score,Per Student Budget,% Overall Passing
,,,,,,
Charter,96.586489,93.620830,83.896421,83.473852,599.500000,95.103660
District,80.799062,66.548453,80.966636,76.956733,643.571429,73.673757
